In elixir, **all code runs inside processes**. Processes are isolated from each other, run concurrent to one another and communicate via message passing. Processes are not only the basis for concurrency in Elixir, but they aslo provide the means for building distributed and fault-tolerant programs.

Elixir's processes should not be confused with operating system processes. Processes in elixir are extrememly lightweight in terms of memory and CPU. Because of this, it is not uncommon to have tens or even hundreds of thousands of processes running simultaneously.

```elixir
spawn fn -> 1 + 2 end
```

```elixir
iex|2 ▶ pid = spawn fn -> 1 + 2 end
#PID<0.137.0>
iex|3 ▶ Process.alive?(pid)
false
```

1. Elixir is an immutable language where nothing is shared by default. If we want to provide buckets, which can be read and modified from multiple places, we have two main options in Elixir:

- Processes
- ETS (Erlang Term Storage)

- **Agent**: Simple wrappers around state
- **GenServer**: "Generic servers"(processes) that encapsulate state, provide sync and async calls, support code reloading, and more. 
- 


In [1]:
self()

#PID<0.209.0>

In [2]:
Process.alive?(self())

true

We can send messages to a process with `send/2` and receive them with `receive/1`

In [3]:
send self(), {:hello, "world"}

{:hello, "world"}

In [ ]:
receive do
 {:hello, msg} -> msg
 {:world, msg} -> "won't match"
end

When a message is sent to a process, the message is stored in the process mailbox. The `receive/1` block goes through the current process mailbox searching for a message that matches any of the given patterns.`receive/1` supports guards and many clauses, such as `case/2`

The process that sends the message does not block on `send/2`, it puts the message in the recipient's mailbox and continues. In particular, a process can send message to itself.

If there is no message in the mailbox matching any of the patterns, the current process will wait until a matching message arrives. A timeout can also be specified:

In [1]:
receive do
  {:hello, msg} -> msg
after
  1_000 -> "nothing after 1s"
  end

"nothing after 1s"

In [2]:
parent = self()
spawn fn -> send(parent, {:hello, self()}) end
receive do
  {:hello, pid} -> "Got hello from #{inspect pid}"
end


"Got hello from #PID<0.224.0>"

`flush/0` flushes and prints all the messages in the mailbox

In [3]:
send self(), :hello

:hello

In [4]:
flush()

:ok

In [5]:
spawn fn -> raise "oops" end

#PID<0.229.0>

[error] Process #PID<0.58.00> raised an exception
** (RuntimeError) oops
    (stdlib) erl_eval.erl:668: :erl_eval.do_apply/6

It merely logged an error but the parent process is still running. That's because processes are isolated. If we want the failure in one process to propagate to another one, we should link them. This can be done with `spawn_link/1`

In [6]:
self()

#PID<0.209.0>

In [ ]:
spawn_link fn -> raise "oops" end

Because processes are linked, we now see a message saying the parent process, which is the shell process, has received an EXIT signal from another process causing the shell to terminate. IEx detects this situation and starts a new shell session

Processes and links play an important role when building fault-tolerant systems. **Elixir processes are isolated and don't share anything by defaut**. Therefore, a failure in a process will never crash or corrupt the state of another process. Links, however, allow proceses to establish a relationship in a case of failures. We often link our processes to supervisors which will detect when a process dies and start a new process in its place.

While other languages would require us to catch/handle exceptions, in Elixir we are actually fine with letting processes fail because we expect supervisors to properly restart our systems. "Failing fast" is a common philosophy when writing Elixir software.

Task build on top of the spawn functions to provide better error reports and introspection:

In [1]:
Task.start fn -> raise "oops" end

{:ok, #PID<0.11096.0>}

We can write process that loop infinitely, maintain state, and send and receive messages. As an example, Let's write a module that starts new processes that work as a key-value store in a file named `kv.exs`:

In [2]:
defmodule KV do
  def start_link do
    Task.start_link(fn -> loop(%{}) end)
  end
  defp loop(map) do
    receive do
      {:get, key, caller} ->
       send caller, Map.get(map, key)
       loop(map)
       {:put, key, value} ->
       loop(Map.put(map, key, value))
       end
  end
end

{:module, KV, <<70, 79, 82, 49, 0, 0, 6, 68, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 194, 0, 0, 0, 21, 9, 69, 108, 105, 120, 105, 114, 46, 75, 86, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, ...>>, {:loop, 1}}

In [3]:
{:ok, pid} = KV.start_link

{:ok, #PID<0.11134.0>}

In [4]:
send pid, {:get, :hello, self()}

{:get, :hello, #PID<0.209.0>}

In [5]:
flush()

:ok

In [6]:
send pid, {:put, :hello, :world}

{:put, :hello, :world}

In [7]:
send pid, {:get, :hello, self()}

{:get, :hello, #PID<0.209.0>}

In [8]:
flush()

:ok

In [9]:
Process.register(pid, :kv)

true

In [10]:
send :kv, {:get, :hello, self()}

{:get, :hello, #PID<0.209.0>}

In [11]:
flush()

:ok

In [13]:
{:ok, file} = File.open "hello", [:write]

{:ok, #PID<0.12253.0>}

That happens because the `IO` module actually works with process. When you write `IO.write(pid, binary)`, the `IO` module will send a message to the process identified by `pid` with the desired operation. Let's see what happens if we use our process

In [12]:
pid = spawn fn ->
  receive do: (msg -> IO.inspect msg)
end

#PID<0.12220.0>

In [13]:
IO.write(pid, "hello")

ErlangError: 1

After `IO/write2`， we can see the request sent by `IO` module(a four-element tuple) printed out.Soon after that, we see that it fails since the `IO` module expected some kind of result we did not supply.

By modeling IO devices with process, the Erlang VM allows different nodes in the same network to exchange file processes in order to read/write files in between nodes. Of all IO devices, there is one that is special to each process: the **group leader**.

When you write `:stdio`, you are actually sending a message to the group leader, which writes to the standard-output file descriptor

In [14]:
IO.puts :stdio, "hello"

hello


:ok

In [15]:
IO.puts Process.group_leader, "hello"

hello


:ok